In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
using Statistics
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/disturbances.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")

In [ ]:
#load borders
#data, keys = import_json_numerics("../data/test/star30.json")
#border = 1000*data["border"];

border = import_border("../data/borders/border.json");

In [ ]:
# create the lattice grid
dx = 10 
Nx, Ny, xrange, yrange, isinside, isborder, n, isgrid = get_grid(border, Float64(dx))

#isgrid = isinside .| isborder
isout = .!isinside .& .!isborder

# define x and y coordinate for plotting
#x = repeat(reshape(xrange,1,Nx), Ny, 1)
#y = repeat(reshape(yrange,Ny,1), 1, Nx);
x = repeat(reshape(1:Nx,1,Nx), Ny, 1)
y = repeat(reshape(1:Ny,Ny,1), 1, Nx);
xr = 1:Nx
yr = 1:Ny
#minimum(n[:,3].^2 + n[:,4].^2)

In [ ]:
#plot(border[:,1], border[:,2])
#scatter(x[isinside], y[isinside], label="inside", markersize=3)
#scatter!(x[isborder], y[isborder], label="boundary", markersize=3)
#scatter!(x[isout], y[isout],label="ouside", markersize=3)

scatter(2x[isinside], 2*y[isinside], label="inside", markersize=2)
scatter!(2*x[isborder], 2*y[isborder], label="boundary", markersize=2)
#scatter!(2*x[isout], 2*y[isout],label="ouside", markersize=3)
#scatter!(2*x[isgrid], 2*y[isgrid],label="ouside", markersize=2)
#scatter!([2*60], [2*28],label="ouside", markersize=4)
#quiver!(vec(2*xr[Int64.(n[:,2])]), vec(2*yr[Int64.(n[:,1])]),
#    quiver=(8*vec(n[:,4]), 8*vec(n[:,3])), linewidth=0.5)

savefig("figure_b.pdf")

In [ ]:
bx = 2 * ones(Ny, Nx)
by = 2 * ones(Ny, Nx)

# adding
#bx .+= local_disturbance(isinside, xrange, yrange, [-200., 250.], 4E6, 500.)
#by .+= local_disturbance(isinside, xrange, yrange, [-200., 250.], 4E6, 500.)

#no static injection
p = zeros(Ny, Nx)

# one "punctual" generator and one load
#p = local_disturbance(isinside, xrange, yrange, [-200.0, -0.0], 9.0, 100.)
#p .+= local_disturbance(isinside, xrange, yrange, [200.0, -0.0], -9.0, 100.)
#contour(p, fill=true)

In [ ]:
# compute the stable solution
th_ref, ths = compute_stable_sol(isinside, n, bx, by, p, Niter=14000, interval= 1000, tol=1E-9)
contour(th_ref, fill=true)

In [ ]:
# evolution for debugging
#ctr_movie(Float64.(1:size(ths,3)), ths)
m = 1E-5*ones(Ny, Nx)
d = 0.6*copy(m)

m[.!isgrid] .= 0.0
d[.!isgrid] .= 0.0;

In [ ]:
# perform a dynamical simulation
# adding a fault
#p = local_disturbance(isinside, xrange, yrange, [750.0, -250.0], -9.0, 100.)
p = local_disturbance(isinside, xrange, yrange, [0.0, 0.0], -9.0, 100.)

#p = local_disturbance(isinside, xrange, yrange, [100.0, -100.], -9.0, 100.)
ts, thetas, omegas = perform_dyn_sim(isinside, n, bx, by, p, m, d, th_ref, interval = 1000, Ndt = Int64(5E4), dt = 0.0001);

In [ ]:
# plotting the frequency at different locations
coord = [0 -250; 0.0 0.0; -500 500; 500. 500.; -1500.0 -1000.0; 750.0 -1000]
#coord = [-250.0 -0; 0.0 0.0; 250 -250; 50. -480.]
time_plot(ts, omegas, coord, tstart = 0.0, tend = 50.0, borders = [border])
current()
#savefig("freq.pdf")


In [ ]:
# evolution of the frequency
ctr_movie(ts, omegas, tstart = 0.0, tend = 50.0)

In [ ]:
# post-fault frequency
#println(sum(omegas[isinside,end])/sum(isinside))
#println(sum(omegas[isgrid,end])/sum(isgrid))
println(sum(p)/sum(d))
println(sum(p[isgrid])/sum(d[isgrid]))
println(sum(p[isinside])/sum(d[isinside]))
#trapz((yrange,xrange),p)/trapz((yrange,xrange),d) 